In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/pneumonia-dataset

/content/drive/MyDrive/pneumonia-dataset


In [20]:
from IPython.display import Image, display
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
import cv2

In [21]:
""" Global variables """
h = 224
w = 224

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

"""Directory to store files"""
create_dir('file')

""" Hyperparameters """
batch_size = 16
lr = 1e-4
num_epochs = 50
model_path = os.path.join("files", "model.h5")
csv_path = os.path.join("files", "log.csv")

In [23]:
train_directory = '/content/drive/MyDrive/pneumonia-dataset/train'
test_directory = '/content/drive/MyDrive/pneumonia-dataset/test'
validation_directory = '/content/drive/MyDrive/pneumonia-dataset/val'

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   rotation_range=10,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_directory,
    target_size=(h, w),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    test_directory,
    target_size=(h, w),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

validation_generator = validation_datagen.flow_from_directory(
    validation_directory,
    target_size=(h, w),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(h, w, 3))

model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))


callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-8, verbose=1),
        CSVLogger(csv_path),
        EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=False),
    ]



model.compile(optimizer=Adam(lr), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()



Found 2000 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d_3  (None, 512)               0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_6 (Dense)             (None, 256)               131328    
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_7 (Dense)             (None, 1)                 257       
                                                              

In [24]:
history = model.fit(train_generator,
                    epochs=50,
                    validation_data=validation_generator,
                    verbose=1,
                    callbacks=callbacks)


Epoch 1/50
125/125 [==============================] - ETA: 0s - loss: 0.4389 - accuracy: 0.8095
Epoch 1: val_loss improved from inf to 0.40139, saving model to files/model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


125/125 [==============================] - 76s 532ms/step - loss: 0.4389 - accuracy: 0.8095 - val_loss: 0.4014 - val_accuracy: 0.8375 - lr: 1.0000e-04
Epoch 2/50
125/125 [==============================] - ETA: 0s - loss: 0.2895 - accuracy: 0.8880
Epoch 2: val_loss did not improve from 0.40139
125/125 [==============================] - 70s 560ms/step - loss: 0.2895 - accuracy: 0.8880 - val_loss: 0.4263 - val_accuracy: 0.8400 - lr: 1.0000e-04
Epoch 3/50
125/125 [==============================] - ETA: 0s - loss: 0.2467 - accuracy: 0.9120
Epoch 3: val_loss improved from 0.40139 to 0.34922, saving model to files/model.h5
125/125 [==============================] - 68s 542ms/step - loss: 0.2467 - accuracy: 0.9120 - val_loss: 0.3492 - val_accuracy: 0.8725 - lr: 1.0000e-04
Epoch 4/50
125/125 [==============================] - ETA: 0s - loss: 0.1977 - accuracy: 0.9300
Epoch 4: val_loss improved from 0.34922 to 0.31275, saving model to files/model.h5
125/125 [==============================] - 70s

In [25]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy}')

25/25 [==============================] - 81s 3s/step - loss: 0.1208 - accuracy: 0.9600
Test accuracy: 0.9599999785423279


In [66]:
def processing(path):
  img = cv2.imread(path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img, (w, h))
  img = img / 255
  img = np.expand_dims(img, axis=0)
  return img


In [70]:
test = "/content/drive/MyDrive/pneumonia-dataset/test/NORMAL/test-normal_049.jpg"
x = model.predict(processing(test))
x = np.squeeze(x, axis=0)
x = 1 if x>.9 else 0
x


1/1 [==============================] - 0s 50ms/step


0